In [4]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import username_n_password  # Import thông tin đăng nhập từ file 'username_n_password' trong cùng folder làm việc
from rich import print # Import rich để hiển thị HTML đẹp!
from rich.syntax import Syntax

In [5]:
# Dictionary để lưu trữ dữ liệu sách
book_dict = {
    'name': [],
    'price': [],
    'category': [],
    'stars': [],
    'upc': [],
    'availability': [],
    'in_stock': [],
    'image_link': []
}

# Dictionary để chuyển đổi số sao từ chữ sang số
number_dict = {'One': '1', 'Two': '2', 'Three': '3', 'Four': '4', 'Five': '5'}

# Tạo thư mục output nếu chưa tồn tại
output_folder = 'output'
book_images_folder = os.path.join(output_folder, 'book_images')
os.makedirs(output_folder, exist_ok=True)  # Tạo folder output
os.makedirs(book_images_folder, exist_ok=True)  # Tạo folder output/book_images

# Tạo file .txt để lưu link ảnh trong thư mục output
image_links_file = os.path.join(output_folder, 'image_links.txt')
with open(image_links_file, 'w') as f:
    pass  # File rỗng ban đầu

# Fake User-Agent để tránh bị chặn
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}

In [6]:
# Gửi Request Có Headers Đến Web Và Xử Lý Pagination (Nhiều Trang)
'''
url = 'https://books.toscrape.com/'
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')
'''
# Đọc danh sách proxy từ file proxy_list.txt và loại bỏ dòng trống
with open('proxy_list.txt', 'r') as p:
    proxy_list = [proxy.strip() for proxy in p if proxy.strip()]

# Chỉ định URL của website đích:
url = "https://books.toscrape.com"

# Lặp qua từng proxy để gửi request
for proxy in proxy_list:
    my_proxy = proxy
    proxy_url = f"http://{my_proxy}"  # Tạo URL của proxy

    proxies = {
        "http": proxy_url,
        "https": proxy_url
    }

    # Thiết lập try-except nâng cao để kiểm tra trạng thái và xử lý lỗi
    try:
        response = requests.get(url, proxies=proxies, headers=headers, timeout=20) # Gửi request với timeout và headers
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        response.raise_for_status() # Nếu gặp lỗi HTTP (403, 500, ...) thì raise lỗi
        # Kiểm tra kết quả
        if response.status_code == 200:
            print(f"Request thành công. Proxy: {proxy}")
        else:
            print(f"Request thất bại. Proxy: {proxy} - Mã trạng thái: {response.status_code}")

    except requests.exceptions.ProxyError: # Xử lý lỗi proxy không hoạt động
        print(f"Proxy lỗi: {proxy}. Thử proxy khác...")

    except requests.exceptions.ConnectTimeout: # Xử lý lỗi timeout khi proxy quá chậm
        print(f"Proxy quá chậm: {proxy}. Bỏ qua...")

    except requests.exceptions.ConnectionError: # Xử lý lỗi khi proxy không thể kết nối đến server
        print(f"Không thể kết nối: {proxy}. Thử proxy khác...")

    except requests.exceptions.HTTPError as err: # Xử lý lỗi HTTP khác (403, 404, 500,...)
        print(f"HTTP Error: {err}")

    except Exception as e: # Xử lý các lỗi khác không xác định
        print(f"Lỗi khác: {e}")
'''
# Hiển trị HTML được làm đẹp bằng .prettify() và rich
rich_html = Syntax(soup.prettify(), "html", theme="solarized-dark", line_numbers=True)
print(rich_html)
'''
# Xác định vị trí, tách chuỗi để lấy tổng số page:
page_count_string = soup.find('li', class_='current').text
page_count = int(page_count_string.strip().split(' ')[-1])

Request thành công. Proxy: 113.160.133.32:8080

In [7]:
# Lặp Qua Từng Trang Để Lấy Dữ Liệu Từng Cuốn Sách (ví dụ bắt đầu từ trang 49)
for page_no in range(49, page_count + 1):
    print(f'Please wait, still scraping page --> {page_no}')
    page_url = f'https://books.toscrape.com/catalogue/page-{page_no}.html'
    response = requests.get(page_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    books = soup.find_all('article', class_='product_pod')

    for book in books:
        book_url = 'https://books.toscrape.com/catalogue/' + book.find('a')['href']
        response = requests.get(book_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Tên sách
        name = soup.find('div', class_='product_main').h1.text
        book_dict['name'].append(name)

        # Giá sách
        price = soup.find('div', class_='product_main').p.text
        book_dict['price'].append(price)

        # Danh mục
        category = soup.find('ul', class_='breadcrumb').find_all('li')[2].a.text
        book_dict['category'].append(category)

        # Số sao
        star_string = soup.find('p', class_='star-rating')['class'][1]
        stars = number_dict[star_string]
        book_dict['stars'].append(stars)

        # Mã UPC
        upc = soup.find('th', string='UPC').find_next_sibling().text
        book_dict['upc'].append(upc)

        # Tình trạng còn hàng
        availability = soup.find('th', string='Availability').find_next_sibling().text
        book_dict['availability'].append(availability)

        # Số lượng còn lại trong kho
        in_stock = availability.split('(')[1].split(' ')[0]
        book_dict['in_stock'].append(in_stock)

        # Link ảnh sản phẩm
        image_link = 'https://books.toscrape.com/' + soup.find('div', class_='thumbnail').img['src'][6:]
        book_dict['image_link'].append(image_link)

        # Lưu link ảnh vào file text trong `output/`
        with open(image_links_file, 'a') as f:
            f.write(image_link + '\n')

Please wait, still scraping page --> 49

Please wait, still scraping page --> 50

In [8]:
# Lưu dữ liệu sách vào file CSV trong thư mục `output/`
csv_file_path = os.path.join(output_folder, 'book_data.csv')
df = pd.DataFrame(book_dict)
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

print('Hoàn thành: xuất file CSV vào {csv_file_path}')

Hoàn thành: xuất file CSV vào {csv_file_path}

In [6]:
# Tải ảnh về và lưu vào `output/book_images/`
with open(image_links_file, 'r') as f:
    links_list = f.read().strip().split('\n')

for i, image_url in enumerate(links_list):
    print(f'Please wait, still scraping image --> {image_url}')
    if image_url:
        response = requests.get(image_url, headers=headers)
        image_path = os.path.join(book_images_folder, f'{i+1}.jpg')
        with open(image_path, 'wb') as imagefile:
            imagefile.write(response.content)
            
print('Hoàn thành: tải file JPEG vào {book_images_folder}')

Please wait, still scraping image --> https://books.toscrape.com/media/cache/7f/9b/7f9bc0563de554998e2d93b7edec4921.jpg
Please wait, still scraping image --> https://books.toscrape.com/media/cache/aa/55/aa55bbf7037a0e19ae69485adcc11903.jpg
Please wait, still scraping image --> https://books.toscrape.com/media/cache/4a/58/4a58f6feb21b7194911eddb020766583.jpg
Please wait, still scraping image --> https://books.toscrape.com/media/cache/a0/bb/a0bb0881c8ee6f5866804cf0a437853f.jpg
Please wait, still scraping image --> https://books.toscrape.com/media/cache/48/7d/487d4be44c60cac7f0085242c4512653.jpg
Please wait, still scraping image --> https://books.toscrape.com/media/cache/2e/fa/2efa13faabc899f61c2a181ab0de7ef8.jpg
Please wait, still scraping image --> https://books.toscrape.com/media/cache/99/f5/99f511d2d38ef356701341d405a1cca2.jpg
Please wait, still scraping image --> https://books.toscrape.com/media/cache/90/f7/90f79652caecac36bc97bf7b769c8fc4.jpg
Please wait, still scraping image --> ht